# Points of Dispense
#### Minimize total distance and vary number of PODs
Maxwell Kennady, Nora Murray, Elizabeth Speigle

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

### Optimization

Read in data from ...

In [2]:
distances = pd.read_csv('data/OD_Pairs_Distances.csv')
population = pd.read_excel('data/BG_master.xlsx')
bg_included = pd.read_csv('data/bg_pop_included.csv')
pod_data = pd.read_csv('data/POD_ids.csv')

In [3]:
pod_data['capacity_50'] = pod_data['student_pop'] * 50

In [4]:
dist_miles = distances.pivot(index='block_group', columns='pod_id', values='Miles')
dist_time = distances.pivot(index='block_group', columns='pod_id', values='TravelTime')

In [5]:
dist = dist_miles.values                     # can change to dist_time
N = population['population'].values          # can change to num_hhs for households
prop = bg_included['all_included'].values    # can change to random_included or other column
cap = pod_data['capacity_50'].values         # can change to other values of capacity

Create indices for block groups and PODs

In [6]:
blocks = range(len(N))
pods = range(len(dist[0]))

Vary number of PODs to open

In [7]:
pods_open = range(10,48)
pods_open_results = {}

In [8]:
for i in pods_open: 
    ### Initialize model for POD locations
    m = gp.Model('POD_locations')
    
    ### Add decision variables
    x = m.addVars(pods, vtype=GRB.BINARY, name='x')
    y = m.addVars(blocks, pods, vtype=GRB.BINARY, name='x')
    
    ### Set objective to minimize total distance across the population
    obj = gp.quicksum(dist[j,i] * x[i] * y[j, i] * N[j] * prop[j] 
                  for j in blocks for i in pods)
    m.setObjective(obj, GRB.MINIMIZE)
    
    ### Add constraints
    #  y[i,j] can only be 1 if x[i] is also 1, meaning POD i is opened
    m.addConstrs((y[j, i] <= x[i] for i in pods for j in blocks), name='y_if_x')
    
    # each block group must be assigned one shelter
    m.addConstrs((gp.quicksum(y[j, i] for i in pods) == 1
             for j in blocks), name='all_blocks_assigned')
    
    # number of PODs opened must be less than num_pods
    m.addConstr((gp.quicksum(x[i] for i in pods) <= i), name='pods_opened')
    
    # capacity at each pod
    # m.addConstrs((gp.quicksum(N[j] * x[i] * y[j, i] for j in blocks) <= cap[i]
    #        for i in pods), name='Capacity')
    
    ### Optimize
    print('Solving for:', i, 'PODs')
    m.optimize()
    
    ### Analysis
    block_pod_list = [[j,i] for j in blocks for i in pods if y[j, i].x==1]
    pods_open_results[i] = block_pod_list

Using license file C:\Users\Elizabeth\gurobi.lic
Academic license - for non-commercial use only
Solving for: 10 PODs
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 52801 rows, 51747 columns and 155147 nonzeros
Model fingerprint: 0x58099f96
Model has 51324 quadratic objective terms
Variable types: 0 continuous, 51747 integer (51747 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+01, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve time: 0.19s
Presolved: 104125 rows, 103071 columns, 309119 nonzeros
Variable types: 0 continuous, 103071 integer (103071 binary)
Found heuristic solution: objective 8779820.8352

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 4.999905e+06, 6124 iterations, 1.01 seconds

    Nodes    |    Current Node

In [9]:
pods_open_total = open('pods_open_total','wb') 
pickle.dump(pods_open_results, pods_open_total)                      
pods_open_total.close()